In [1]:
%matplotlib inline

import time
import datetime as dt
import os
import glob
import json

import itertools as it

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

os.getcwd()

'/home/edmz/Data Covid/Code'

In [2]:
from dicts_paths_v3 import *
from calc_functions_v3 import *
# from graph_functions_v1 import *

Today is 11/04/2021. Retrieve data from how many days? 0
Ok, will retrieve data from 11/04/2021


https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/

https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-personnes-vaccinees-contre-la-covid-19-1/

# dep -> dep-tot-3C-hebd

In [3]:
dataset = 'sp-pos-quot-dep'
data_fname, path_temp = data_path(date_choice, dataset, version = '5Lv3')
din = pd.read_csv(data_fname, sep = ';', parse_dates = ['jour'], dtype = {'dep': str})
din.sample(3)

../Data/sp-pos-quot-dep-2021-04-11-18h07.csv


,dep,jour,P,T,cl_age90,pop
308988,84,2021-02-22,1,42,90,7890.0
92354,27,2020-09-10,0,16,90,7264.0
119485,32,2021-02-07,1,8,39,18813.0


In [4]:
d = din.copy()
d = d.rename(columns = {'dep': 'entity'})
d = columns_first(d, columns = ['entity', 'cl_age90', 'jour'])
d_dep = d

d = din.copy()
d['reg'] = d['dep'].map(lambda x: dep2reg[x])
d = d.drop(columns = ['dep'])
dg = groupby_sum(d, columns = ['reg', 'jour', 'cl_age90'])
dg = d.rename(columns = {'reg': 'entity'})
dg = columns_first(dg, columns = ['entity', 'cl_age90', 'jour'])
d_reg = dg

d = din.copy()
d = d.drop(columns = ['dep'])
dg = groupby_sum(d, columns = ['jour', 'cl_age90'])
dg['entity'] = 'France entière'
dg = columns_first(dg, columns = ['entity', 'cl_age90', 'jour'])
d_nat = dg

d = pd.merge(d_dep, d_reg, how = 'outer')
d = pd.merge(d, d_nat, how = 'outer')
d = columns_first(d, columns = ['entity', 'cl_age90', 'jour'])
d.sample(3)

,entity,cl_age90,jour,P,T,pop
727997,Île-de-France,69,2021-03-21,11,97,121128.0
371885,977,89,2020-06-27,0,0,181.0
279857,76,69,2021-02-22,77,813,152712.0


### dep-tot -> dep-tot-3C

In [5]:
d['three_class'] = d['cl_age90'].map(lambda x: class_2_3C[x])
d = d.drop(columns = 'cl_age90')
dg = groupby_sum(d,['entity', 'jour', 'three_class'])
dg = columns_first(dg, columns = ['entity', 'three_class', 'jour'])
d = dg
d.sample(3)

,entity,three_class,jour,P,T,pop
128075,971,whole,2021-01-10,2,95,376879.0
44175,33,whole,2020-09-10,434,6753,1633440.0
8138,07,60+,2020-06-30,0,88,107028.0


### tot-3C ->tot-3C-hebd

In [6]:
dg = (d
      .groupby(['entity', 'three_class'])
      .rolling(window = 7, on = 'jour')
      .sum()
      .fillna(0)
      .reset_index()
      .set_index('level_2')
      .rename(columns = {
          'P': 'P hebdo',
          'T': 'T hebdo',
              })
     )
d[['P hebdo', 'T hebdo']] = dg[['P hebdo', 'T hebdo']]
d['incidence hebdo'] = d['P hebdo'] / d['pop'] * 100000
d['positivite hebdo'] = d['P hebdo'] / d['T hebdo'] * 100
d['taux de test hebdo'] = d['T hebdo'] / d['pop'] * 100000
d.sample(3)

,entity,three_class,jour,P,T,pop,P hebdo,T hebdo,incidence hebdo,positivite hebdo,taux de test hebdo
128520,972,0-29,2020-06-05,0,88,108501.0,2.0,279.0,1.843301,0.716846,257.140487
86260,65,0-29,2020-07-02,0,34,64264.0,2.0,161.0,3.112162,1.242236,250.529068
18721,15,30-59,2020-06-28,0,1,52528.0,0.0,145.0,0.000000,0.000000,276.043253


In [7]:
fname = path_temp + '-tot-3C-hebd.csv'
d.to_csv(fname, sep = ';', index = False)

### fra, reg, dep  -> fra-3C, reg-3C, dep-3C

In [8]:
# dataset = 'sp-pos-quot-fra'
# fname, path_temp, = data_path(date_choice, dataset, version = '5Lv3')
# din = pd.read_csv(fname, sep = ';', parse_dates = ['jour'], dtype = {})
# din.sample(3)

In [9]:
# d = din.copy()
# d = d.rename(columns = {'fra': 'entity'})
# d['three_class'] = d['cl_age90'].map(lambda x: class_2_3C[x])
# d = d.drop(columns = 'cl_age90')
# dg = groupby_sum(d,['entity', 'jour', 'three_class'])
# dg = columns_first(dg, columns = ['entity', 'three_class', 'jour'])
# d = dg
# d.sample(3)

In [10]:
# dg = (d
#       .groupby(['entity', 'three_class'])
#       .rolling(window = 7, on = 'jour')
#       .sum()
#       .fillna(0)
#       .reset_index()
#       .set_index('level_2')
#       .rename(columns = {
#           'P': 'P hebdo',
#           'T': 'T hebdo',
#               })
#      )
# d[['P hebdo', 'T hebdo']] = dg[['P hebdo', 'T hebdo']]
# d['incidence hebdo'] = d['P hebdo'] / d['pop'] * 100000
# d['positivite hebdo'] = d['P hebdo'] / d['T hebdo'] * 100
# d['taux de test hebdo'] = d['T hebdo'] / d['pop'] * 100000
# d.sample(3)

In [11]:
# dout = d
# fname = path_temp + '-3C-hebd.csv'
# dout.to_csv(fname, sep = ';', index = False)

In [12]:
# dataset = 'sp-pos-quot-reg'
# fname, path_temp, = data_path(date_choice, dataset, version = '5Lv3')
# din = pd.read_csv(fname, sep = ';', parse_dates = ['jour'], dtype = {'reg': str})
# din.sample(3)

In [13]:
# d = din.copy()
# d = d.rename(columns = {'reg': 'entity'})
# d['three_class'] = d['cl_age90'].map(lambda x: class_2_3C[x])
# d = d.drop(columns = 'cl_age90')
# dg = groupby_sum(d,['entity', 'jour', 'three_class'])
# dg = columns_first(dg, columns = ['entity', 'three_class', 'jour'])
# d = dg
# d.sample(3)

In [14]:
# dg = (d
#       .groupby(['entity', 'three_class'])
#       .rolling(window = 7, on = 'jour')
#       .sum()
#       .fillna(0)
#       .reset_index()
#       .set_index('level_2')
#       .rename(columns = {
#           'P': 'P hebdo',
#           'T': 'T hebdo',
#               })
#      )
# d[['P hebdo', 'T hebdo']] = dg[['P hebdo', 'T hebdo']]
# d['incidence hebdo'] = d['P hebdo'] / d['pop'] * 100000
# d['positivite hebdo'] = d['P hebdo'] / d['T hebdo'] * 100
# d['taux de test hebdo'] = d['T hebdo'] / d['pop'] * 100000
# d.sample(3)

In [15]:
# dout = d
# fname = path_temp + '-3C-hebd.csv'
# dout.to_csv(fname, sep = ';', index = False)

In [16]:
# dataset = 'sp-pos-quot-dep'
# fname, path_temp, = data_path(date_choice, dataset, version = '5Lv3')
# din = pd.read_csv(fname, sep = ';', parse_dates = ['jour'], dtype = {'dep': str})
# din.sample(3)

In [17]:
# d = din.copy()
# d = d.rename(columns = {'dep': 'entity'})
# d['three_class'] = d['cl_age90'].map(lambda x: class_2_3C[x])
# d = d.drop(columns = 'cl_age90')
# dg = groupby_sum(d,['entity', 'jour', 'three_class'])
# dg = columns_first(dg, columns = ['entity', 'three_class', 'jour'])
# d = dg
# d.sample(3)

In [18]:
# dg = (d
#       .groupby(['entity', 'three_class'])
#       .rolling(window = 7, on = 'jour')
#       .sum()
#       .fillna(0)
#       .reset_index()
#       .set_index('level_2')
#       .rename(columns = {
#           'P': 'P hebdo',
#           'T': 'T hebdo',
#               })
#      )
# d[['P hebdo', 'T hebdo']] = dg[['P hebdo', 'T hebdo']]
# d['incidence hebdo'] = d['P hebdo'] / d['pop'] * 100000
# d['positivite hebdo'] = d['P hebdo'] / d['T hebdo'] * 100
# d['taux de test hebdo'] = d['T hebdo'] / d['pop'] * 100000
# d.sample(3)

In [19]:
# fname = path_temp + '-3C-hebd.csv'
# dout.to_csv(fname, sep = ';', index = False)

# Traitement des données hospitalières

In [20]:
dataset = 'donnees-hospitalieres-classe-age-covid19'
fname, path_temp, = data_path(date_choice, dataset, version = '5Lv3')
din = pd.read_csv(fname, sep = ';', parse_dates = ['jour'], dtype = {'reg': int})
din.sample(3)

../Data/donnees-hospitalieres-classe-age-covid19-2021-04-11-18h02.csv


,reg,cl_age90,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
39578,93,0,2020-10-03,798,167,NaN,NaN,NaN,8307,1162
30189,28,49,2020-08-17,1,0,NaN,NaN,NaN,153,11
26194,11,29,2020-07-28,45,4,NaN,NaN,NaN,844,12


In [21]:
d = din.copy()
d['entity'] = (d['reg']
               .map(lambda x: reg_name[str(x)] )
               .replace({
                   'Guadeloupe':'Outre-mer',
                   'Martinique':'Outre-mer',
                   'Guyane':'Outre-mer',
                   'La Réunion':'Outre-mer',
                   'Mayotte':'Outre-mer',
               })
              )
d = columns_first(d, columns = ['entity'])
d = d.drop(columns = ['reg'])
d.head(3)

,entity,cl_age90,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
0,Outre-mer,0,2020-03-18,0,0,NaN,NaN,NaN,0,0
1,Outre-mer,9,2020-03-18,0,0,NaN,NaN,NaN,0,0
2,Outre-mer,19,2020-03-18,0,0,NaN,NaN,NaN,0,0


In [22]:
d['three_class'] = d['cl_age90'].map(lambda x: class_2_3C[x])
d = d.drop(columns = 'cl_age90')
dg = groupby_sum(d,['entity', 'jour', 'three_class',])
dg = columns_first(dg, ['entity', 'three_class', 'jour',])
d_reg = dg
d_reg.tail(3)

,entity,three_class,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
21837,Île-de-France,30-59,2021-04-11,1997,639,1147.0,165.0,46.0,25166,1314
21838,Île-de-France,60+,2021-04-11,5734,1093,2855.0,1750.0,36.0,49248,16061
21839,Île-de-France,whole,2021-04-11,8023,1769,4217.0,1937.0,100.0,79792,17522


In [23]:
d = d_reg.copy()
dg = groupby_sum(d, columns = ['jour', 'three_class',])
dg['entity'] = 'France entière'
dg = columns_first(dg, columns = ['entity', 'three_class', 'jour',])
d_nat = dg
d_nat.sample(3)

,entity,three_class,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
151,France entière,whole,2020-04-24,28554,4785,0.0,0.0,0.0,43486,13841
803,France entière,whole,2020-10-04,6964,1335,0.0,0.0,0.0,98343,21425
534,France entière,60+,2020-07-29,4331,240,0.0,0.0,0.0,52874,18292


In [24]:
d_tot = pd.merge(d_reg, d_nat, how = 'outer')
d_tot = d_tot.drop(columns = ['HospConv', 'SSR_USLD', 'autres', 'rad'])
d = d_tot
d.sample(3)

,entity,three_class,jour,hosp,rea,dc
15017,Occitanie,30-59,2020-11-17,233,68,45
2531,Bourgogne-Franche-Comté,whole,2020-11-15,1776,237,1541
19982,Provence-Alpes-Côte d'Azur,60+,2021-01-27,2885,312,4229


In [25]:
d['dc hebdo'] = d['dc'] - (d.groupby(['entity', 'three_class'])
                              .shift(7)
                             )['dc']


d['taux hosp'] = d.apply(lambda x: x['hosp'] / reg_3C_pop 
                                                    [ x['entity'] ]
                                                    [ x['three_class'] ] * 100000, 
                         axis = "columns")
d['taux rea'] = d.apply(lambda x: x['rea'] / reg_3C_pop
                                                    [ x['entity'] ]
                                                    [ x['three_class'] ] * 100000, 
                        axis = "columns")
d['taux décès'] = d.apply(lambda x: x['dc hebdo'] / reg_3C_pop
                                                    [ x['entity'] ]
                                                    [ x['three_class'] ] * 100000, 
                        axis = "columns")
d.sample(10)

,entity,three_class,jour,hosp,rea,dc,dc hebdo,taux hosp,taux rea,taux décès
7339,Corse,whole,2020-12-17,21,6,113,0.0,6.092625,1.740750,0.000000
2063,Bourgogne-Franche-Comté,whole,2020-07-21,59,6,1061,3.0,2.119985,0.215592,0.107796
15688,Outre-mer,0-29,2020-04-09,5,0,0,0.0,0.522536,0.000000,0.000000
17387,Pays de la Loire,whole,2020-05-13,544,48,388,20.0,14.309023,1.262561,0.526067
7962,Grand Est,60+,2020-04-27,3524,450,2652,252.0,232.940296,29.745498,16.657479
3784,Bretagne,0-29,2020-08-31,1,0,0,0.0,0.090095,0.000000,0.000000
9051,Grand Est,whole,2021-01-24,3035,324,6892,307.0,55.064211,5.878354,5.569922
2381,Bourgogne-Franche-Comté,30-59,2020-10-09,19,4,39,1.0,1.842737,0.387945,0.096986
1173,Auvergne-Rhône-Alpes,30-59,2021-01-05,419,73,188,4.0,13.579220,2.365831,0.129635
7309,Corse,30-59,2020-12-10,6,3,8,1.0,4.385997,2.192998,0.730999


In [26]:
dout = d
fname = path_temp + '-3C-hebd.csv'
dout.to_csv(fname, sep = ';', index = False)

# Traitement des données VAC-SI 

In [27]:
dataset = 'vacsi-a-dep'
fname, path_temp = data_path(date_choice, dataset, version = '5Lv3')
din = pd.read_csv(fname, sep = ';', parse_dates = ['jour'], dtype = {'dep': str})
din.sample(3)

../Data/vacsi-a-dep-2021-04-11-18h15.csv


,dep,clage_vacsi,jour,n_dose1,n_dose2,n_cum_dose1,n_cum_dose2
71218,66,79,2021-01-24,80,0,1920,2
13037,12,59,2021-01-13,85,0,214,0
105719,972,69,2021-03-26,77,16,1454,502


In [28]:
d = din.copy()
d = d[(d.dep != '00') & (d.dep != '750') & (d.dep != '970')] 
d = d.rename(columns = {'dep': 'entity'})
d = columns_first(d, columns = ['entity', 'clage_vacsi', 'jour'])
d_dep = d

d = d_dep.copy()
d['entity'] = d['entity'].map(lambda x: dep2reg[x])
dg = groupby_sum(d, columns = ['entity', 'jour', 'clage_vacsi'])
d_reg = dg

d = d_dep.copy()
dg = groupby_sum(d, columns = ['jour', 'clage_vacsi'])
dg['entity'] = 'France entière'
d_nat = dg

d = pd.merge(d_dep, d_reg, how = 'outer')
d = pd.merge(d, d_nat, how = 'outer')
d = columns_first(d, columns = ['entity', 'clage_vacsi', 'jour'])
d.sample(3)

,entity,clage_vacsi,jour,n_dose1,n_dose2,n_cum_dose1,n_cum_dose2
3351,04,29,2021-04-02,6,0,134,21
60172,57,49,2021-01-03,0,0,0,0
103160,973,39,2021-02-15,4,0,138,10


In [29]:
d['three_class'] = d['clage_vacsi'].map(lambda x: classvac_2_3C[x])
d = d.drop(columns = 'clage_vacsi')
dg = groupby_sum(d,['entity', 'jour', 'three_class'])
dg = columns_first(dg, columns = ['entity', 'three_class', 'jour'])
d = dg
d.tail(10)

,entity,three_class,jour,n_dose1,n_dose2,n_cum_dose1,n_cum_dose2
49550,Île-de-France,60+,2021-04-08,25784,15876,1114622,409488
49551,Île-de-France,whole,2021-04-08,42433,18093,1572436,506339
49552,Île-de-France,0-29,2021-04-09,1930,187,52953,7016
49553,Île-de-France,30-59,2021-04-09,17171,1992,423456,91925
49554,Île-de-France,60+,2021-04-09,29572,15847,1144194,425335
49555,Île-de-France,whole,2021-04-09,48694,18029,1621130,524368
49556,Île-de-France,0-29,2021-04-10,975,187,53928,7203
49557,Île-de-France,30-59,2021-04-10,9920,2121,433376,94046
49558,Île-de-France,60+,2021-04-10,17000,15665,1161194,441000
49559,Île-de-France,whole,2021-04-10,27907,17976,1649037,542344


In [30]:
d['taux dose 1'] = d.apply(lambda x: x['n_cum_dose1'] / reg_3C_pop 
                                                    [ x['entity'] ]
                                                    [ x['three_class'] ] * 100, 
                         axis = "columns")
d['taux dose 2'] = d.apply(lambda x: x['n_cum_dose2'] / reg_3C_pop
                                                    [ x['entity'] ]
                                                    [ x['three_class'] ] * 100, 
                        axis = "columns")
d.sample(3)

,entity,three_class,jour,n_dose1,n_dose2,n_cum_dose1,n_cum_dose2,taux dose 1,taux dose 2
11067,28,whole,2021-02-01,324,160,10788,457,2.512197,0.106421
38145,90,30-59,2021-03-23,99,4,4332,1784,8.148678,3.355780
7353,18,30-59,2021-02-18,29,108,3184,1756,2.935536,1.618970


In [31]:
dout = d
fname = path_temp + '-tot-3C.csv'
dout.to_csv(fname, sep = ';', index = False)